In [ ]:
import torch
from EEResnet18 import ResNet, data_loader, ResidualBlock
from tqdm import tqdm

In [ ]:
test_loader = data_loader(data_dir='./data',
                              batch_size=1,
                              test=True, data_model="cifar10")

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

device = torch.device(device)

In [ ]:
model = ResNet().to(device)
model.make_backbone(ResidualBlock, [2, 2, 2, 2], [1, 3], 10)
model.set_inference_parameters(True, 0.5)
model.load_state_dict(torch.load("model/RESNET_18-CIFAR_10-SGD-Early_Exits_on_1st_and_3rd_layer-Cuda.pth", map_location=device, weights_only=True))
model.eval()
total_times = 0
total_step = len(test_loader)
correct = 0
total = 0
with tqdm(total=total_step) as bar_step:
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs, times_of_exit = model(images)
        total_times += times_of_exit
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs
        if (i + 1) % 3 == 0 or i == total_step + 1:
            bar_step.set_postfix({
                            'Times that image exited earlier': total_times,
                            # 'avg_accuracy': total_accuracy / (i + 1),
                            # 'max_abs_gradient': np.max(abs(model.weight.grad))
                        })
            cur_n_batch = i % 3 + 1
            bar_step.update(cur_n_batch)

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))